In [ ]:
from pathlib import Path
import pandas as pd
import json

# Укажи свой путь до папки с датасетом
base_dir = Path("C:/Users/user/Desktop/slovo")
csv_path = base_dir / "annotations.csv"
json_dir = base_dir / "json"
json_dir.mkdir(parents=True, exist_ok=True)

# Загрузка CSV с аннотациями
df = pd.read_csv(csv_path)

# Разделение на train и dev по полю 'train'
train_data = {}
dev_data = {}

for _, row in df.iterrows():
    record = {
        "name": row['attachment_id'],
        "text": row['text'],
        "video_path": f"{row['attachment_id']}.mp4",
        "pose": f"{row['attachment_id']}.pkl"
    }
    if row['train']:
        train_data[row['attachment_id']] = record
    else:
        dev_data[row['attachment_id']] = record

# Сохранение в JSON
with open(json_dir / "slovo_train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open(json_dir / "slovo_dev.json", "w", encoding="utf-8") as f:
    json.dump(dev_data, f, ensure_ascii=False, indent=2)

print("JSON-файлы сохранены в:", json_dir)


In [ ]:
from types import SimpleNamespace

args = SimpleNamespace(
    dataset="CSL_News",         # заглушка: влияет только на структуру путей в S2T_Dataset_news
    task="SLT",                 # задача — перевод жестов в текст (Sign Language Translation)
    output_dir="C:/Users/user/Desktop/slovo/output",  # куда сохранять модель и логи
    finetune="",
    batch_size=4,
    num_workers=2,
    pin_mem=True,               # ускорение передачи данных на GPU
    seed=42,                    # фиксируем сид для воспроизводимости
    distributed=False,         # отключаем распределённое обучение
    hidden_dim=256,            # размер скрытого признакового пространства
    max_length=32,             # максимальная длина последовательности кадров (T)
    rgb_support=False,         # работаем только с позами (без RGB)
    gradient_accumulation_steps=2,
    gradient_clipping=1.0,
    offload=False,             # не используем offload
    dtype="bf16",
    zero_stage=2,
    quick_break=0,
    warmup_epochs=0,
    epochs=10,
    eval=False,
    label_smoothing=0.1,
    lr=1e-4,
    min_lr=1e-6
)


In [ ]:
from pathlib import Path
from datasets import S2T_Dataset_news

# Пути к аннотациям
base_dir = Path("C:/Users/user/Desktop/slovo")
train_json = base_dir / "json/slovo_train.json"
dev_json = base_dir / "json/slovo_dev.json"

# Загрузка датасетов
train_dataset = S2T_Dataset_news(train_json, args, phase="train")
dev_dataset = S2T_Dataset_news(dev_json, args, phase="dev")

print(f"Размер обучающей выборки: {len(train_dataset)}")
print(f"Размер валидационной выборки: {len(dev_dataset)}")


In [ ]:
from torch.utils.data import DataLoader, SequentialSampler

train_loader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=True,                           #обязательно перемешать train
    num_workers=args.num_workers,
    collate_fn=train_dataset.collate_fn,   #кастомная функция объединения батча
    pin_memory=args.pin_mem,
    drop_last=True                         #убрать последний неполный батч
)

dev_loader = DataLoader(
    dev_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(dev_dataset),  # без перемешивания для валидации
    num_workers=args.num_workers,
    collate_fn=dev_dataset.collate_fn,
    pin_memory=args.pin_mem
)


In [ ]:
from timm.optim import create_optimizer
from transformers import get_scheduler

# Создаём оптимизатор для модели
optimizer = create_optimizer(args, model)

# Планировщик learning rate с косинусным спадом
lr_scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=args.epochs * len(train_loader),
)


In [ ]:
from pathlib import Path
import time
import datetime
import json
from models import get_requires_grad_dict
import utils

output_dir = Path(args.output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

max_bleu = 0  # максимальный BLEU-4 по ходу обучения
start_time = time.time()


ModuleNotFoundError: No module named 'models'

In [ ]:
from pre_training import train_one_epoch, evaluate

for epoch in range(args.epochs):
    # Обучение на одной эпохе
    train_stats = train_one_epoch(args, model, train_loader, optimizer, epoch, model_without_ddp=model)

    # Оценка BLEU-4 на dev-наборе
    test_stats = evaluate(args, dev_loader, model, model)

    # Если метрика BLEU улучшилась — сохраняем чекпойнт
    if test_stats["bleu4"] > max_bleu:
        max_bleu = test_stats["bleu4"]
        best_ckpt = output_dir / "best_checkpoint.pth"
        utils.save_on_master({"model": get_requires_grad_dict(model)}, best_ckpt)

    # Выводим BLEU-4 для текущей эпохи
    print(f"Epoch {epoch} — BLEU-4: {test_stats['bleu4']:.2f} — Max BLEU-4: {max_bleu:.2f}")


In [ ]:
    log_stats = {
        **{f'train_{k}': v for k, v in train_stats.items()},
        **{f'test_{k}': v for k, v in test_stats.items()},
        'epoch': epoch,
        'n_parameters': utils.count_parameters_in_MB(model)
    }

    with (output_dir / "log.txt").open("a") as f:
        f.write(json.dumps(log_stats) + "\n")


In [ ]:
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print("Общее время обучения:", total_time_str)


In [ ]:
from torch.utils.data import DataLoader, SequentialSampler
from datasets import S2T_Dataset_news

# Повторно определим dev-набор
dev_dataset = S2T_Dataset_news(dev_json, args, phase="dev")
dev_loader = DataLoader(
    dev_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(dev_dataset),
    num_workers=args.num_workers,
    collate_fn=dev_dataset.collate_fn,
    pin_memory=args.pin_mem
)


In [ ]:
from models import Uni_Sign
import torch

# Заново инициализируем модель и загружаем лучшие веса
model = Uni_Sign(args).cuda()
checkpoint = torch.load(output_dir / "best_checkpoint.pth", map_location="cuda")
model.load_state_dict(checkpoint["model"], strict=False)
model.eval()


In [ ]:
from sklearn.metrics import classification_report

assert len(tgt_refs) == len(tgt_pres)

y_true = [ref.strip().lower() for ref in tgt_refs]
y_pred = [pred.strip().lower() for pred in tgt_pres]

classes = sorted(list(set(y_true) | set(y_pred)))

# Генерация отчёта
print("Classification Report:\n")
print(classification_report(y_true, y_pred, labels=classes, digits=2))


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

y_true = [ref.strip().lower() for ref in tgt_refs]
y_pred = [pred.strip().lower() for pred in tgt_pres]

all_classes = sorted(list(set(y_true) | set(y_pred)))

# Строим матрицу ошибок
cm = confusion_matrix(y_true, y_pred, labels=all_classes)
df_cm = pd.DataFrame(cm, index=all_classes, columns=all_classes)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Вывод матрицы
print("\nConfusion Matrix:\n")
print(df_cm.to_string())
